In [2]:
#!/usr/bin/env python
# coding: utf-8
# Python program to generate Defect Clustering
# Developed By: Priyanka Mukhopadhyay & Sriram Srinivasan

# importing all necessery modules 
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 
import pandas as pd 
import collections

# natural language processing: n-gram ranking
import re
import unicodedata
import nltk
from nltk.corpus import stopwords

#imports to create pdf
from fpdf import FPDF

#imports to create UI
from tkinter import *
from tkinter import filedialog 
from tkinter import ttk
from tkinter.filedialog import asksaveasfilename
from PIL import ImageTk, Image
from tkinter import messagebox
import os
import csv

def wordCloudAnalyser(filePath,filterColumn,wordGram,stpWords,getFreq,varWordCloud,barChart):
    
    print("File Name: "+filePath+"\nWordGram: ",wordGram,"\nStop words from user: "+stpWords+"\nFilter Frequency: ",getFreq)
    
    try:
    
        nltk.download('stopwords')

        nltk.download('wordnet')

        stopWordsFromUser = list(stpWords.split(","))

        ADDITIONAL_STOPWORDS = ["defect", "incident", "mismatch","matching","instead","code","populated","issue","internal","summary",
                                "delimited","column","count","getting","generated","displayed","error","please","table"]

        def basic_clean(text): 
          #print("iteration",1)  
          #print(type(text))
          #print(text)
          """
          function to clean up the data. All the words that
          are not designated as a stop word is then lemmatized after
          encoding and basic regex parsing performed.
          """
          wnl = nltk.stem.WordNetLemmatizer()
          #::added STOPWORDS also
          stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS + stopWordsFromUser + list(STOPWORDS)
          text = (unicodedata.normalize('NFKD', text)
            .encode('ascii', 'ignore')
            .decode('utf-8', 'ignore')
            .lower())
          words = re.sub(r'[^\w\s]', '', text).split()
          
          words = [''.join([character.strip() for character in word if character.isalnum() or character =='_' ]) for word in words if not word.isnumeric()]
          return list(set([wnl.lemmatize(word) for word in words if word not in stopwords and not word.isnumeric()] ))

        df = pd.read_csv(filePath, encoding ="latin-1")
        #filter null/nan values
        print("Filtering null/nan values")
        df=df.dropna(subset=[filterColumn])
        df[filterColumn] = df[filterColumn].astype(str)
        df['Cleaned'] = df[filterColumn].apply(basic_clean)
        df['bigrams'] = df['Cleaned'].apply(lambda row: list(nltk.ngrams(row, wordGram))) # Applying ngrams lamba fn
        
        comment_words = [] 
        stopwords = set(STOPWORDS)

        # iterate through the csv file 
        for val in df.bigrams:
            #::Using___ to differentiate 2 words of the bigram
            c=['___'.join((b)) for b in val]
            comment_words.append(c)

        comment_words=str(comment_words)   
        #::removing quotes
        comment_words=comment_words.replace("'","")
        #print(comment_words)

        wordcloud = WordCloud(font_path='DejaVuSans.ttf',width = 1000, height = 1000, 
                        background_color='black', colormap='Set2',collocations=False, 
                        #stopwords = stopwords, 
                        min_font_size = 4).generate(comment_words) 
        print("Wordcloud loaded")

        # plot the WordCloud image                        
        fig1 = plt.figure(figsize = (8,8), facecolor = None)
        plt.title('Intelligent Incident Predictor'+ "\n\n",fontsize=25, 
                  color="black",loc="center",fontweight="bold")
        print("Title defined")
        plt.imshow(wordcloud) 
        plt.axis("off") 
        plt.tight_layout(pad = 0) 
        plt.savefig(varWordCloud)
        print("Created Word Cloud")
        #plt.show()
        print("Plotted WordCloud")

        comment_words_list = list(comment_words.replace('[', '').replace(']', '').replace(' ', '').split(","))
        #remove empty values from list
        comment_words_list = list(filter(None, comment_words_list))
        print("Empty values removed from list.....")
        counter=collections.Counter(comment_words_list)
        grouped_df = pd.DataFrame.from_dict(counter, orient='index').reset_index().rename(columns={'index':'Words', 0:'Count'}).sort_values('Count',ascending=False)
        print("Filtering Data....")

        if getFreq > 0:
            filteredDf = grouped_df.loc[grouped_df['Count'] >= getFreq]
        else:
            filteredDf = grouped_df

        if filteredDf.empty:
                print("Dataframe is empty!!")
                return "EmptyDataframe"
        else:
                print("Generating Bar Graph")
                ax = filteredDf.plot.bar(x="Words",y="Count",align='center')
                plt.title('Incident Frequency Chart'+ "\n\n",fontname="DejaVu Sans",fontsize=20, 
                          color="black",loc="center",fontweight="bold")
                print("Title defined")
                plt.xlabel(filterColumn)
                plt.ylabel('Frequency')
                plt.rcParams['figure.dpi'] = 200
                for p in ax.patches:
                    ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005),ha='left', va='bottom',fontsize=5)
                plt.savefig(barChart,bbox_inches='tight')
                #plt.show(block=True)
                print("Created bar graph")
                return "SUCCESS"
    except ValueError as ve:  
        print(ve)
        message="ValueError- Try one of the following:\n1)Choose a non-date column\n2)Choose a non-empty column\n3)Choose a lower word gram if the column is non-date and non-empty"
        return message
    
    except Exception as e:
        print(e)
        return str(e)

def saveAsPdf(targetPath,logo_cts,logo_cust,wordcloud,barchart):
    try:
        class PDF(FPDF):



            def footer(self):
                # Position at 3.2 cm from bottom
                self.set_y(-32)
                # Arial italic 8
                self.set_font('Arial', 'I', 8)
                # Text color in gray
                self.set_text_color(128)
                # Page number
                self.cell(0, 10, 'Page ' + str(self.page_no()), 0, 0, 'C')
                self.image(logo_cts,x = 170, y = 265, w = 20, h = 10, type = 'PNG', link = '')
                self.image(logo_cust,x = 20, y = 248, w = 30, h = 45, type = 'PNG', link = '')


        pdf = PDF()
        #pdf.set_title(title)
        pdf.add_page()
        pdf.image(wordcloud,x = 5, y = 30, w = 200, h = 200, type = 'PNG', link = '')
        print("Adding image: "+wordcloud)

        pdf.add_page()
        pdf.image(barchart,x = 5, y = 30, w = 200, h = 200, type = 'PNG', link = '')
        print("Adding image: "+barchart)

        pdf.output(targetPath, 'F')
        return "SUCCESS"
    except Exception as e:
        print(str(e))
        return str(e)

#UI code
filename=""
logo_cognizant="cts_logo.png"
logo_customer="customer_logo.png"
wordcloud="wordcloud.png"
barchart ="barchart.png"

def browseFiles():
    global filename
    filename = filedialog.askopenfilename(initialdir = "/", 
                                          title = "Select a File", 
                                          filetypes = (("CSV Files","*.csv"),("CSV Files","*.csv"))) 
    if not filename.strip():
        
        messagebox.showinfo("Alert!!!", "Please choose a file to proceed")
        label_file_explorer.configure(text=filename+"No file chosen...")
    
    else:  
        # Change label contents 
        label_file_explorer.configure(text=filename+"\nUploaded Successfully")
        updateDropdown()

                                                                                                 
# Create the root window 
window = Tk() 
   
# Set window title 
window.title("Intelligent Incident Detector") 
   
# Set window size 
window.geometry("1500x600")
   
#Set window background color 
window.config(background = "black")

# Create left and right frames
left_frame = Frame(window, width=200, height= 600, bg='black')
left_frame.grid(row=0, column=0, padx=10, pady=10)

right_frame = Frame(window, width=1200, height=600, bg='black')
right_frame.grid(row=0, column=1, padx=10, pady=10)

right_label=Label(right_frame,text = "Display Panel",width = 30, height = 4,  
                            fg = "white",bg="black",font=("Arial", 44,"italic bold"))
right_label.grid(column = 0, row = 0,sticky=N+W+E)

right_sub_label=Label(right_frame,text = "(Results will appear here. Please maximize the window for better view)",width = 100, height = 2,  
                            fg = "white",bg="black",font=("Arial", 15,"italic bold"))
right_sub_label.grid(column = 0, row = 0,sticky=S+W+E)

# Create a File Explorer label 
label_file_explorer = Label(left_frame,  
                            text = "Choose file to upload\nFile should be in csv format\n(Eg: example.csv)", 
                            width = 50, height = 4,  
                            fg = "white",bg="black") 
   
       
button_explore = Button(left_frame,  
                        text = "Browse...", 
                        command = browseFiles)  
 

label_file_explorer.grid(column = 0, row =0) 
   
button_explore.grid(column = 0, row = 1,sticky=N+S+E+W,padx=25) 

#get column list from csv files
def updateDropdown():
        f= open("%s" %filename, 'r')
        d_reader = csv.DictReader(f)
        headers = d_reader.fieldnames
        colsToChoose['values'] = headers 
        

#create label for column drop down
Label(left_frame, text = "Choose a column to analyse\nNote: This column will be used to create word gram",  
          width = 50, height = 2,fg = "white",bg="black").grid(column =0, row = 5,padx=5, pady=5) 

#column drop down
col = StringVar(left_frame,"Select a column") 
colsToChoose = ttk.Combobox(left_frame, width = 27, justify=CENTER, textvariable = col) 
colsToChoose.grid(column =0, row = 6,sticky=N) 
colsToChoose.current() 

#Create label for radio button
label_radio = Label(left_frame,text = "Choose word gram",width = 50, height = 2,  
                            fg = "white",bg="black")

label_radio.grid(column =0, row = 10,padx=5, pady=5) 


#create radio buttons
v = IntVar(left_frame, 1) 

# Dictionary to create multiple buttons 
values = {"1 Word" : 1, 
          "2 Words" : 2, 
          "3 Words" : 3, 
          "4 Words" : 4} 
  
# Loop is used to create multiple Radiobuttons 
# rather than creating each button separately 
for (text, value) in values.items(): 
    Radiobutton(left_frame, text = text, variable = v, 
        value = value,bg="black",activebackground="white",fg="white",selectcolor="grey").grid(column=0)


#Create text box

Label(left_frame,text = "Enter words to be filtered separated by comma\n(Eg: hello,world)",width = 40, height = 2,  
                            fg = "white",bg="black").grid(column = 0, row = 19,sticky=S)


textBox = StringVar(left_frame, "Enter Text...")
textEntered = ttk.Entry(left_frame, width = 30, textvariable = textBox,justify = CENTER,
                       font=("Arial", 10,"italic"),foreground="grey")
textEntered.grid(column = 0, row = 20,sticky=N)

#create double click event  for text box
def double_click1(event): 
    textEntered.delete("0","end")
    textEntered.config(foreground="black",font=("Arial", 10,"bold"))

textEntered.bind('<Double 1>', double_click1)

#create frequency slider
v1 = IntVar()

Label(left_frame,text = "Choose a frequency between 0 to 100\nwith slider",width = 30, height = 2,  
                            fg = "white",bg="black").grid(column = 0, row = 25,sticky=S)

Scale( left_frame, variable = v1,from_ = 0, to = 100,  
           orient = HORIZONTAL,
     width=10,resolution=1,sliderlength=20).grid(column= 0, row = 27,sticky=S) 

#adding option for manual entry of frequency
Label(left_frame,text = "Or\nEnter frequency manually\n(Eg: 105)",width = 30, height = 5,  
                            fg = "white",bg="black").grid(column = 0, row = 30, pady=5)
freqBox = StringVar(left_frame, "0")
freqEntered = ttk.Entry(left_frame, width = 5, textvariable = freqBox,justify = CENTER,
                       font=("Arial", 10),foreground="grey")
freqEntered.grid(column = 0, row = 30,sticky=S)

#create double click event for frquency box
def double_click2(event): 
    freqEntered.delete("0","end")
    freqEntered.config(foreground="black",font=("Arial", 10,"bold"))

freqEntered.bind('<Double 1>', double_click2)

def diaplayResult():
    
    if not filename.strip():
        
        messagebox.showinfo("Alert!!!", "No File Found for Analysis")
    
    else:
        filterCol = col.get()
        if filterCol == "Select a column":
            messagebox.showinfo("Alert!!!", "Please choose a column to analyse")          
        else:
            print(filename)
            wordGram = v.get()
            stpWords = textBox.get()
            freqFromScale=v1.get()

            #check if text box is empty
            if bool(freqBox.get().strip())==False:
                freqFromBox=0
            else:
                freqFromBox=int(freqBox.get())


            if freqFromScale>0 and freqFromBox==0:       
                getFreq = freqFromScale
            elif freqFromBox>0 and freqFromScale==0:
                getFreq = freqFromBox
            elif freqFromBox>0 and freqFromScale>0:
                result = messagebox.askquestion("Alert!!!",'To proceed with manually entered frequency {} press "Yes"'.format(freqFromBox)+
                                   '\nTo proceed with frequency slider value {} press "No"'.format(freqFromScale), icon='warning')
                if result == 'yes':
                    getFreq = freqFromBox 
                else:
                    getFreq = freqFromScale
            else:
                getFreq=0

            msg = wordCloudAnalyser(filename,filterCol,wordGram,stpWords,getFreq,wordcloud,barchart)

            def displayWordCloud(imgWord):
                global img1
                img1 = ImageTk.PhotoImage(Image.open(imgWord).resize((700, 800), Image.ANTIALIAS))
                Label(right_frame, image = img1).grid(column=0,row=2,padx=10, pady=5)


            def displayBarChart(imgBar):
                global img2
                img2 = ImageTk.PhotoImage(Image.open(imgBar).resize((750, 800), Image.ANTIALIAS))
                Label(right_frame, image = img2).grid(column=1,row=2,padx=10, pady=5)

            if msg=="SUCCESS":

                right_label.destroy()
                right_sub_label.destroy()
                displayWordCloud(wordcloud)
                displayBarChart(barchart)

            elif msg=="EmptyDataframe":

                messagebox.showinfo("Alert!!!", "No records found having frequency>={}.\nPlease check with a lower frequency".format(getFreq))

            else:

                messagebox.showinfo("Error!!!", "Exception Caught...."+msg)
        

def exportAsPdf():
    targetPath = asksaveasfilename(initialfile='IncidentDetector',title='SaveFileAs', initialdir='C:\\', filetypes=(("PDF file", "*.pdf*"),),
                                 defaultextension='.pdf')
    msg = saveAsPdf(targetPath,logo_cognizant,logo_customer,wordcloud,barchart)
    
    if msg=="SUCCESS":
        messagebox.showinfo("SUCCESS", "PDF file exported at: "+targetPath)
    else:
        messagebox.showinfo("Error!!!", "Exception Occured..\n{}".format(msg))

Button(left_frame,text = "Analyse & Display",command = diaplayResult).grid(column= 0, row = 40, sticky=E,pady=30)
Button(left_frame,text = "Export As PDF",command = exportAsPdf).grid(column= 0, row = 40, sticky=W,padx=25,pady=30)

#events on closure
def onClosure():
    try:
        if messagebox.askokcancel("Quit","Do you want to quit?"):
            try:
                os.remove(wordcloud)
                print("deleted "+wordcloud)
            except Exception as e:
                print(str(e))
            try:
                os.remove(barchart)
                print("deleted "+barchart)
            except Exception as e:
                print(str(e))
            print("Closing session.....")
            window.destroy()
    except Exception as e:
        print("Oops! "+str(e))
        print("Closing session!!!!!")
        window.destroy()

window.protocol("WM_DELETE_WINDOW",onClosure)

window.mainloop()

[WinError 2] The system cannot find the file specified: 'wordcloud.png'
[WinError 2] The system cannot find the file specified: 'barchart.png'
Closing session.....
